In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


def make_coordinates(image, line_coordinates):
    slope, intercept = line_coordinates
    y1 = image.shape[0]  #y-intercept

    # y = mx + b
    # y2 = line goes three fifth of the way. See lane_edge_detection_optimized_explanation.png
    y2 = int(y1 * (3/5))

    x1 = int((y1- intercept)/slope)
    
    x2 = int((y2- intercept)/slope)

    return np.array([x1,y1,x2,y2])

def average_slope(image, lines):
    left_lines = []
    right_lines = []

    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        
        #polyfit funtion to find the slope and y intercept for each line in a form of [slope, y-intercept]
        #polyfit(x cooridinates, y cooridninates, degree)   x.shape must be equal to y.shape and degree = 1 indicates to be linear function
        parameters = np.polyfit((x1,x2), (y1,y2),1)
        slope = parameters[0]
        intercept = parameters[1]
        print(parameters)

        if slope < 0: 
            left_lines.append((slope, intercept))
        else:
            right_lines.append((slope, intercept))
    #np.average to return the average slope and intercept for left lanes and right lanes respectively.
    left_average_slope_intercept = np.average(left_lines, axis=0)
    right_average_slope_intercept = np.average(right_lines, axis=0)
    print(left_average_slope_intercept, " : Average slope and intercept for left lane")
    print(right_average_slope_intercept, " : Average slope and intercept for right lane and intercept")

    left_line = make_coordinates(image, left_average_slope_intercept)
    right_line = make_coordinates(image, right_average_slope_intercept)

    return np.array([left_line, right_line])



def cannyFunction(image):
    g_image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    
    #ksize (5,5) could be any odd number. Even number will not be accepted. The higher ksize parameters, The more blurrer the image it is  
    blur_image = cv2.GaussianBlur(g_image, (5, 5), 0)
    
    canny_image = cv2.Canny(blur_image, 50, 150)
    return canny_image

def regionOfInterest(image): 
    height = image.shape[0]

    #specifying the region where we want to isloate from the image
    triangle = np.array([
        [(200, height), (1100, height), (600, 250)]
    ])
    # print(triangle.shape) //output = (1,3,2)
    # print(triangle[0][2][0])
    # triangle[0][0][0] = 200
    # trinagle[0][0][1] = 700 //height of the given image
    # triangle[0][1][0] = 1100
    # triangle[0][1][1] = 700 //height of the given image
    # triangle[0][2][0] = 600
    # triangle[0][2][1] = 250
    mask = np.zeros_like(image) ### important! mask//mask_image need to be set to zeros in order to blend lines_image to original image because bitwise_add() to 0 is equal to zeros    
    #specify the area//polygon needs to be filled in color 255; 
    # mask = image,
    # triangle = Array of polygons where each polygon is represented as an array of points.
    # colorPicker = 255
    cv2.fillPoly(mask, triangle, 255) 
    mask_image = cv2.bitwise_and(image, mask)
    return mask_image
    

def display_lines(image, lines):
    #define a image variable in a form of 3-D array (row, column, colorPicker in heximal and fill with 0 which is black)
    lines_image = np.zeros_like(image) 
    
    if lines is not None: #Null checked
        for line in lines:
            print("Line\'s coordinates:",  line)  
            #e.g [[704 418 927 641]]
            #e.g [[767 493 807 534]]
            #each line is a 2D array containing our line coordinates in a form [[x1, y1, x2, y2]].
            #These coordinates specify the line's parameters, as well as the location of the lines with respect to the image space (in pixel resolution (xxx * yyy)
            x1, y1, x2, y2 = line.reshape(4)

            #(255,0,0) which is blue in ColorPicker in RGB format 
            #1 is the thickness of the lines in the image
            cv2.line(lines_image, (x1, y1), (x2, y2), (255, 0, 0), 10) 

    return lines_image


#read image by calling cv2.imread()
# load_image = cv2.imread("./Image/Image/test_image.jpg")

# #copy load_image variable over copy_image as a multi-dimensional array
# copy_image = np.copy(load_image)

# #Call cannyFuntion to proceed the image for edge detection
# canny_image = cannyFunction(copy_image)

# #Region of interest 
# cropped_image = regionOfInterest(canny_image)

# #Edge detection   
# #minLineLength=40  If traced edge is less than 40 pixel, the line would not be accepted.
# #maxLineGap=5  It indeicates the maximum distance in pixels between segmented lines which we will allow to be connected instead of being broken up
# #e.g  --- --  to be -----      
# edge = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5)

# average_lines = average_slope(copy_image, edge)

# #display_lines function draws lines on a black image which is image variable in a form of 3-D array
# lines_image = display_lines(copy_image, average_lines)



# #Display image with matplotlib.pyplot.imshow() along with x and T axis
# # plt.imshow(canny_image)
# # plt.show()


# #addWeighted(source_image_1, P1, source_image_2, p2, gamma)  
# # All elements in source_image_1 will be mutiplied by P1 to decrease the pixel intensity values in image_1, which makes the image_1 a bit darker (optional).
# # All elements in source_image_2 will be mutiplied by P2 to decrease the pixel intensity values in image_1, in which makes no difference to the original source_image_2 (no need to dark the lines)
# # Lastly gamma will be added to the sum of source_image_1 and source_image_2 
# combined_image = cv2.addWeighted(copy_image, 0.8, lines_image, 1, 1)

# #Display image with applying simple edge detection
# #cv2.imshow("result", combined_image)


# #cv2.imshow("result", cropped_image)
# cv2.waitKey(0)

cap = cv2.VideoCapture("test2.avi")

if(cap.isOpened() == False):
    print("error opening file")


while cap.isOpened():
    _, frame = cap.read()
    canny_image = cannyFunction(frame)

    cropped_image = regionOfInterest(canny_image)
    
    edge = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5)

    average_lines = average_slope(frame, edge)

    lines_image = display_lines(frame, average_lines)

    combined_image = cv2.addWeighted(frame, 0.8, lines_image, 1, 1)

    cv2.imshow("result", combined_image)
    if cv2.waitKey(1) & 0xFF== ord('q'):
        break



[   1. -286.]
[   0.94827586 -254.44827586]
[  -1.87919463 1358.11409396]
[   0.96078431 -268.11764706]
[  -1.675 1246.8  ]
[   1. -287.]
[   0.96666667 -271.5       ]
[  -1.68253968 1248.36507937]
[   0.94202899 -252.1884058 ]
[   1. -288.]
[   0.93589744 -247.51282051]
[  -1.7455781  1284.42639111]  : Average slope and intercept for left lane
[   0.96920666 -269.34589365]  : Average slope and intercept for right lane and intercept
Line's coordinates: [323 720 488 432]
Line's coordinates: [1020  720  723  432]
[   1. -287.]
[   0.92682927 -237.12195122]
[  -1.88157895 1358.96052632]
[   0.94827586 -257.84482759]
[   0.94318182 -254.09090909]
[   1. -288.]
[   0.92857143 -240.71428571]
[   0.92957746 -247.88732394]
[   1. -288.]
[   0.96610169 -275.72881356]
[   0.94339623 -254.66037736]
[  -1.88157895 1358.96052632]  : Average slope and intercept for left lane
[   0.95859338 -263.10484885]  : Average slope and intercept for right lane and intercept
Line's coordinates: [339 720 492 432

[   0.96666667 -266.86666667]
[  -1.88235294 1352.35294118]
[   0.94230769 -245.90384615]
[   0.98181818 -284.45454545]
[  -1.93877551 1374.57142857]
[  -1.90161646 1360.37013875]  : Average slope and intercept for left lane
[   0.98115542 -274.21611762]  : Average slope and intercept for right lane and intercept
Line's coordinates: [336 720 488 432]
Line's coordinates: [1013  720  719  432]
[   0.96666667 -266.86666667]
[   1. -283.]
[   0.96803653 -268.61643836]
[   1. -280.]
[   1. -282.]
[  -1.87619048 1351.27619048]
[   0.96610169 -260.86440678]
[   0.94642857 -249.625     ]
[   0.94520548 -249.83561644]
[   1.0212766  -299.14893617]
[   1.02564103 -301.25641026]
[   1. -281.]
[  -1.93617021 1379.42553191]
[  -1.90618034 1365.3508612 ]  : Average slope and intercept for left lane
[   0.98539605 -274.74667952]  : Average slope and intercept for right lane and intercept
Line's coordinates: [338 720 489 432]
Line's coordinates: [1009  720  717  432]
[   0.96666667 -266.86666667]
[   

[  -1.8742515  1345.74850299]
[   1. -275.]
[   1. -276.]
[  -1.8742515  1345.74850299]  : Average slope and intercept for left lane
[   0.98905169 -271.52344322]  : Average slope and intercept for right lane and intercept
Line's coordinates: [333 720 487 432]
Line's coordinates: [1002  720  711  432]
[   0.96923077 -265.76923077]
[   1. -277.]
[   0.96507937 -260.37142857]
[   1. -275.]
[  -1.8742515  1345.74850299]
[   1. -275.]
[   1. -276.]
[  -1.8742515  1345.74850299]  : Average slope and intercept for left lane
[   0.98905169 -271.52344322]  : Average slope and intercept for right lane and intercept
Line's coordinates: [333 720 487 432]
Line's coordinates: [1002  720  711  432]
[   0.96774194 -262.09677419]
[   1. -277.]
[   1. -275.]
[  -1.73015873 1260.12698413]
[  -1.87341772 1343.82278481]
[   0.93939394 -236.84848485]
[  -1.63157895 1223.42105263]
[  -1.76923077 1293.84615385]
[   0.94 -239.04]
[   1. -274.]
[  -1.87878788 1344.63636364]
[  -1.70833333 1251.58333333]
[   1.

C:\Users\ernes\Anaconda3\lib\site-packages\numpy\lib\function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\ernes\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


TypeError: cannot unpack non-iterable numpy.float64 object